# LAB4 - Introduction to Entity Linking/Named Entity Disambiguation

In this notebook we will introduce the task of Named entity linking/disambiguation. We will cover the following aspects:
1. Task definition
2. Opportunities and challenges
3. The three components of every Entity linking system

### 1. Task definition

#### 1.1 Entity tasks in NLP

**Entity tasks so far** So far, we have seen two tasks that relate to the entities mentioned in text: 
1. recognizing/spotting entity mentions in text in the task of *Named Entity Recognition*
2. classifying these entity mentions to their *type* (for example, Person or City) - this is done in the task of *Named Entity Classification/Typing*

**NED** Here, we will introduce Named Entity Disambiguation - NED, also called (Named) Entity Linking - (N)EL. NED is a central task in information extraction. The goal is to take the entity mentions that were found in text with the task of NER and "disambiguate" them. In this sense, the task of Entity Disambiguation builds on top of the output of the NER task. Sometimes the tasks are combined together in a task called Named Entity Recognition and Disambiguation (NERD).

#### 1.2 Disambiguation

Ok, so WHY do we need to disambiguate entity mentions found in text and HOW do we do that?

**WHY** Let's take an entity mention we find in text, like "JFK". This phrase can mean different things. It can refer to John F. Kennedy (the former American president), or the airport in NYC with the same name , or to "Justice For Khojaly", etc. So, disambiguation is the task to say precisely which of these world entities is the correct one in a specific textual document.

**HOW** To disambiguate, we need a way to map this ambiguous mention found in text to a unique "representation" that already exists which has a clear meaning. Such representations, for example, are the Wikipedia pages of these world entities, because each Wikipedia page has a URL ("http:///wikipedia.org/...") and each URL describes exactly one entity. For example, https://en.wikipedia.org/wiki/John_F._Kennedy_International_Airport  describes the JFK airport and not the president.

Resources like Wikipedia are called *Knowledge Bases (KBs)*, because they contain knowledge about entities in the world. There are two types of knowledge bases: structured and unstructured. Named Entity Disambiguation in practice is sometimes performed with respect to unstructured and sometimes with respect to structured KBs.

Wikipedia is an example for an unstructured knowledge base, because most of its content is in unstructured (running text) form, such as: "John F. Kennedy international airport is a public airport owned by the city of New York ..." 

Examples for structured knowledge bases are DBpedia and Wikidata. In a structured knowledge base, we would not find a textual description, but rather a structured list of facts, such as:

John F. Kennedy International airport
```
* airport type: public
* owner: city of New York
* ...
```

Here is the representation of the JFK airport in these structured knowledge bases:
* http://dbpedia.org/resource/John_F._Kennedy_International_Airport
* https://www.wikidata.org/wiki/Q8685


Note that most entities have information in these three knowledge bases (Wikipedia, DBpedia, and Wikidata). For example, we can find information about the John F. Kennedy airport in Wikipedia (https://en.wikipedia.org/wiki/John_F._Kennedy_International_Airport) or in DBpedia (http://dbpedia.org/resource/John_F._Kennedy_International_Airport). Actually, it is always the case that the last part of the URL is the same across Wikipedia and DBpedia ("John_F.\_Kennedy\_International_Airport" in this case), which is convenient for us to use information from both places if we want.

To summarize, we perform disambiguation of entity mentions in text by connecting them to existing entities in a knowledge base, like Wikipedia. This kind of disambiguation "links" the textual mention to an existing representation - for this reason the task is also called Entity Linking.

#### 1.3 Example
For example, let's consider the following sentence:

"_JetBlue_ begins direct service between _Barnstable Airport_ and _JFK_."

The entity mentions we find here are: "JetBlue", "Barnstable Airport" and "JFK". Let's say that we perform linking to DBpedia. Then, “JetBlue” should be linked to the entity http://dbpedia.org/resource/JetBlue, and “JFK” to http://dbpedia.org/resource/John_F._Kennedy_International_Airport. 

However, there is no entry in DBpedia for the Barnstable Municipal Airport, which is the meaning of the mention “Barnstable Airport”. We cannot link this entity then. The entities for which there is no representation in a chosen knowledge base are called *NIL entities*. When a system processes the text, it should simply say that the meaning of “Barnstable Airport” is _NIL_.

### 2. Opportunities and challenges

**Connecting text and knowledge bases** NED establishes a connection between the information in text and knowledge bases on the external world. Note that these knowledge bases were not created to improve text processing. Instead, they were created independently to provide knowledge about the world - for example, Wikipedia, DBpedia, and Wikidata give us encyclopedic knowledge. 

**Opportunities** By establishing a link between a phrase in text and a unique entry in a knowledge base, we directly get access to much more knowledge that we can use to enhance the information in text. If we know that 'JFK' refers to the airport, we allow our tools to have access to all facts about this airport, such as its location and founding year. In addition, if we want, we can now extract facts from text and store these in the knowledge base.

**Challenges** Why is entity linking not an easy task? This relates to the aspects of ambiguity and variance. 

*Ambiguity* is the number of meanings that a certain entity mention can have. For example, imagine how many people in the world are called "John Smith". DBpedia contains entries for a few hundreds of them, see http://dbpedia.org/page/John_Smith. How can we teach a computer to decide which of these is the one mentioned in text? And, what if the mentioned John Smith is actually a NIL entity and is not stored in DBpedia?

There are also many cases where it is quite easy to link an entity to a knowledge base. Often the mentions in text have little ambiguity (for example, "Barack Obama"). Or, they have multiple meanings but one of them is almost always used: for example, there are multiple cities called "Paris", but the French capital will be most often mentioned in text.

*Variance* is the amount of different mentions that refer to the same entity. For example, http://dbpedia.org/resource/John_F._Kennedy_International_Airport can be called "JFK", or "John F. Kennedy Airport", or "The NYC airport" in text.

### 3. Named Entity Disambiguation in practice: 3 phases

In practice, most NED systems consist of three phases:

1. **Entity recognition/spotting** - this is done as described in the NER(C) task. In the example sentence "_JetBlue_ begins direct service between _Barnstable Airport_ and _JFK_.", the recognition phase will detect the entity mentions: "JetBlue", "Barnstable Airport", and "JFK".
2. **Candidate generation** - here we take each of the recognized mentions and look these up in the knowledge base for potential meanings. For example, the phrase "JFK" could have the following candidates:
    * http://dbpedia.org/resource/John_F._Kennedy
    * http://dbpedia.org/resource/John_F._Kennedy_International_Airport
    * http://dbpedia.org/resource/JFK_(film)
    * http://dbpedia.org/resource/JFK_University
    * http://dbpedia.org/resource/Justice_for_Khojaly
    * etc...

Similar lists will be generated for the other mentions found in text: "JetBlue" and "Barnstable Airport". The candidate generation phase is not trivial because of the ambiguity and variation described above. Also, new entities are appearing all the time in news articles, so the number of options grows over time.

To understand the complexity of this step, compare it to the "candidate" classes in sentiment analysis. In latter task, we perform classification to one of the fixed three categories (positive, neutral, negative) for each piece of text, while for the task of entity linking the number of classes is different for each mention and can sometimes be very large.

3. **Disambiguation** - the goal of this final phase is to take the list of potential meanings generated in the candidate generation phase for each of the mentions and make a decision on which instance is the correct one. This decision can either be: choosing one of the possible candidates, or deciding that none of the candidates is the correct one (NIL entity).

As the list of candidates is different for each mention, it is not easy to perform this disambiguation with supervised learning approaches as in other tasks (e.g., NER). In practice, most systems use different methods; we will briefly describe two methods in the notebook Lab4-Entity-linking-tools.